#### Removing Bad Participants from the Analysis.
1. We are removing the first **n** trials for every cue in every casino. This ensures that the participants are able to learn the pattern of every slot as we want to measure how the RewP changes in different average task values. For that the participants have to learn first what the average task value is of every slot.
2. We are removing participants that do not manage to get **threshold** of the learnable trials correct, i.e. choosing the lever with the higher win probability. Thus, we assume that participants did not truly care about the reward. As we want to measure the RewP in different averaging task values, this requires participants to truly want to win corresponding to dopamine release on a win.

In [1]:
import os
import pandas as pd

# HYPERPARAMS
n = 4
threshold = 0.7

subjects_dir = 'Dataset/ds004147'
subject_dirs = [d for d in os.listdir(subjects_dir) if os.path.isdir(os.path.join(subjects_dir, d))]

for subject_dir in subject_dirs:
    beh_file_path = os.path.join(subjects_dir, subject_dir, 'beh', f'{subject_dir}_task-casinos_beh.tsv')

    if os.path.exists(beh_file_path):
        df = pd.read_csv(beh_file_path, sep='\t')
        df_sorted = df.sort_values(by=['cue', 'block'])
        # remove the first four trials for every cue and block
        filtered_df = df_sorted.groupby(['cue', 'block']).apply(lambda x: x.iloc[4:]).reset_index(drop=True)
        # aggregate learnable trials and mark participants that did not meet the threshold
        learnable_trials = filtered_df[filtered_df['prob'] == 80]
        correct_choices = learnable_trials['outcome'] == learnable_trials['optimal']
        success_rate = correct_choices.mean()
        threshold_met = success_rate >= threshold
        if threshold_met:
            print(f"{subject_dir} does meet the threshold and should be included in the analysis.")
            filtered_df.to_csv(os.path.join(subjects_dir, subject_dir, 'beh', f'{subject_dir}_task-casinos_beh_filtered.tsv'), sep='\t', index=False)
        else:
            print(f"{subject_dir} does  not meet the threshold and should be not included in the analysis.")
            filtered_df.to_csv(os.path.join(subjects_dir, subject_dir, 'beh', f'{subject_dir}_REJECT_task-casinos_beh_filtered.tsv'), sep='\t', index=False)
    else:
        print(f'Behavioral file for {subject_dir} does not exist.')

Behavioral file for .git does not exist.
Behavioral file for code does not exist.
sub-27 does meet the threshold and should be included in the analysis.
sub-28 does meet the threshold and should be included in the analysis.
sub-29 does meet the threshold and should be included in the analysis.
sub-30 does meet the threshold and should be included in the analysis.
sub-31 does meet the threshold and should be included in the analysis.
sub-32 does meet the threshold and should be included in the analysis.
sub-33 does meet the threshold and should be included in the analysis.
sub-34 does meet the threshold and should be included in the analysis.
sub-35 does meet the threshold and should be included in the analysis.
sub-36 does meet the threshold and should be included in the analysis.
sub-37 does meet the threshold and should be included in the analysis.
sub-38 does meet the threshold and should be included in the analysis.


After choosing n=4 and threshold=0.7, no participants went under the threshold. The authors noted that some were excluded, so we believe that these participants where from the other test site.

#### Re-Referencing:

The authors re-referenced to the average of the mastoid signals, using the "TP9" and "TP10" channels. 
Since this is a common rereferencing scheme, we have used the same, by setting the relevant parameter in the config file as follows:
```eeg_reference = ['TP9', 'TP10'] ``` .

#### Downsampling:

To save on processing power required, we have downsampled the data from 1000 Hz to 150 Hz. mne-bids-pipeline constructs the PSD plot for frequencies up to 1.5 * lowpass_cutoff_frequency. Thus 150 Hz is sufficient to allow us to include all required frequencies in our analysis.